<a href="https://colab.research.google.com/github/Taniyasumbul/CyberWise/blob/main/copy_of_llm_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle --quiet
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Drive path set karo
file_path = '/content/drive/MyDrive/cybersecurity-qa-v1.csv'

# File load karo
df = pd.read_csv(file_path)

# Sample 5000 rows
if len(df) > 5000:
    df_small = df.sample(5000, random_state=42)
else:
    df_small = df

# Save small CSV
df_small.to_csv('/content/drive/MyDrive/cybersecurity-qa-small.csv', index=False)

print("✅ New small file saved in Drive!")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ New small file saved in Drive!


In [ ]:
def format_conversation(example):
    conversation = f"<|user|>: {example['input_text']}\n<|assistant|>: {example['output_text']}"
    return {"text": conversation}

In [ ]:
print(df.columns)

Index(['question', 'answer'], dtype='object')


In [ ]:
df.rename(columns={'question': 'input_text'}, inplace=True)
df.rename(columns={'answer': 'output_text'}, inplace=True)


In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
# Convert the applied result to a DataFrame
formatted_dataset = df.apply(format_conversation, axis=1).tolist()
formatted_df = pd.DataFrame(formatted_dataset)

# Now convert to Hugging Face Dataset
from datasets import Dataset
hf_dataset = Dataset.from_pandas(formatted_df)

print(hf_dataset)


Dataset({
    features: ['text'],
    num_rows: 42484
})


In [ ]:

# Assuming formatted_dataset is a list of dictionaries like [{'text': "..."}]
formatted_df = pd.DataFrame(formatted_dataset)

# Convert DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(formatted_df)

# Check column names
print(hf_dataset.column_names)



['text']


In [ ]:
from transformers import AutoTokenizer


# Initialize tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assign pad token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     padding="max_length",
                     truncation=True,
                     max_length=1024)

# Make sure you're using the Hugging Face Dataset, not the list
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

print(tokenized_dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/42484 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 42484
})


In [ ]:
# Add labels field to tokenized dataset
def add_labels(examples):
    examples["labels"] = examples["input_ids"]
    return examples

# Map the function over your dataset
tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)


Map:   0%|          | 0/42484 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.reset_format()  # Reset format to allow modification
tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

Map:   0%|          | 0/42484 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",

    eval_steps=1000,
    save_steps=1000,
    logging_steps=500,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    fp16=True,  # If you're on GPU (Colab GPU works)
    max_steps=1000,  # Optional if you want shorter runs
    report_to="wandb"
)


In [ ]:
def add_labels(example):
    example["labels"] = example["input_ids"].clone()
tokenized_dataset = tokenized_dataset.map(add_labels)


Map:   0%|          | 0/42484 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Step,Training Loss
500,1.611300
1000,1.628700


TrainOutput(global_step=1000, training_loss=1.6199798583984375, metrics={'train_runtime': 635.9686, 'train_samples_per_second': 12.579, 'train_steps_per_second': 1.572, 'total_flos': 2090374004736000.0, 'train_loss': 1.6199798583984375, 'epoch': 0.18830618585820544})

In [ ]:
trainer.save_model("cybercrime-llm-final")
tokenizer.save_pretrained("cybercrime-llm-final")


NameError: name 'trainer' is not defined

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("cybercrime-llm-final")
tokenizer = AutoTokenizer.from_pretrained("cybercrime-llm-final")


OSError: cybercrime-llm-final is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
input_text = "What is cyber crime?"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is cyber crime?

I am a student in a university. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science. I am studying computer science and computer science.


In [ ]:
model_path = "/content/cybercrime-llm-final"  # or use your Drive path


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)



In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/drive/MyDrive/cybersecurity-qa-small.csv")  # replace with your actual file
df['text'] = df['question'] + "\n" + df['answer']


dataset = Dataset.from_pandas(df[['text']])


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cybersecurity-qa-small.csv'

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./cybercrime_continue_training",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_dir="./logs",
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    warmup_steps=10,
    fp16=True,
    resume_from_checkpoint=model_path  # 🚀 This continues from where you stopped
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sumbultaniya (sumbultaniya-kiet-group-of-institutions) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.954100
100,3.772100
150,3.771000
200,3.685700
250,3.703900
300,3.743800
350,3.726900
400,3.650300
450,3.719200
500,3.771800


TrainOutput(global_step=25000, training_loss=3.1225276196289062, metrics={'train_runtime': 2222.2615, 'train_samples_per_second': 22.5, 'train_steps_per_second': 11.25, 'total_flos': 3266209382400000.0, 'train_loss': 3.1225276196289062, 'epoch': 10.0})

In [ ]:
trainer.train(resume_from_checkpoint=True)


NameError: name 'trainer' is not defined

In [ ]:
trainer.save_model("my_finetuned_model")
tokenizer.save_pretrained("my_finetuned_model")


NameError: name 'trainer' is not defined

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="my_finetuned_model", tokenizer="my_finetuned_model")
result = pipe("What is cyber crime?", max_length=100)
print(result[0]['generated_text'])


KeyboardInterrupt: 

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./cybercrime_continue_training",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    warmup_steps=10,
    fp16=True,
    resume_from_checkpoint=model_path  # 🚀 This continues from where you stopped
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss
50,3.187400
100,3.236500
150,3.270500
200,3.216600
250,3.231800
300,3.286200
350,3.281200
400,3.212800
450,3.305000
500,3.332300


Step,Training Loss
50,3.187400
100,3.236500
150,3.270500
200,3.216600
250,3.231800
300,3.286200
350,3.281200
400,3.212800
450,3.305000
500,3.332300


TrainOutput(global_step=7500, training_loss=3.233254654947917, metrics={'train_runtime': 701.1791, 'train_samples_per_second': 21.393, 'train_steps_per_second': 10.696, 'total_flos': 979862814720000.0, 'train_loss': 3.233254654947917, 'epoch': 3.0})

In [ ]:
# Replace this with your preferred save path
save_directory = "/content/drive/MyDrive/gpt2_cybercrime_model"

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)


('/content/drive/MyDrive/gpt2_cybercrime_model/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2_cybercrime_model/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2_cybercrime_model/vocab.json',
 '/content/drive/MyDrive/gpt2_cybercrime_model/merges.txt',
 '/content/drive/MyDrive/gpt2_cybercrime_model/added_tokens.json',
 '/content/drive/MyDrive/gpt2_cybercrime_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and model
model_path = "/content/drive/MyDrive/gpt2_cybercrime_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Automatically use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Your input
input_text ="Explain cybercrime in one paragraph. Avoid repeating sentences:"


# Tokenize and move inputs to the same device
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print the response
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Explain cybercrime in one paragraph. Avoid repeating sentences:

I am a student and I have been reading about cyber crime in the past. I have been reading about cyber crime in the past. I have been reading about cyber crime in the past. I have been reading about cyber crime in the past. I have been reading about cyber crime in the past. I have been reading about cyber crime in the past. I have been reading about cyber crime in the past. I have been reading about cyber crime in the past. I have been reading about


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/gpt2_saved_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 👇 Prompt style
input_text = "Explain cybercrime in simple terms, in one paragraph only."

inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# 👇 Decoding settings
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Explain cybercrime in simple terms, in one paragraph only.


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="my_finetuned_model", tokenizer="my_finetuned_model")
result = pipe("What is cyber crime?", max_length=100)
print(result[0]['generated_text'])



Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


What is cyber crime?

Do cyber-crime is legal for individuals using computer networks? Is it a legal problem because the threat they're using is limited, like in cyber-criminal networks.
I just came across a new article: How cyber-attacks are able to stop people from copying, replacing and replacing a software or service that has been updated.
Let me say:

a) computers can be copied in a different way
b) computers can be replaced (like in


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("my_finetuned_model")
tokenizer = AutoTokenizer.from_pretrained("my_finetuned_model")


OSError: my_finetuned_model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/cybercrime-llm-final"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/my_finetuned_model"  # This is where it was saved

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/my_finetuned_model'. Use `repo_type` argument if needed.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/gpt2_saved_model"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example prompt
input_text = "What is cybercrime?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is cybercrime?


In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gpt2_saved_model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gpt2_saved_model")

# Continue training as before


In [ ]:
pip install Flask


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/content/drive/MyDrive/gpt2_cybercrime_model"  # your saved model path

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)




In [ ]:
pip install --upgrade sympy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cud

In [ ]:
from flask import Flask, request, jsonify
import torch  # or another framework if not using PyTorch
from transformers import AutoModelForCausalLM, AutoTokenizer  # For Hugging Face models, if you're using them

app = Flask(__name__)

# Load your fine-tuned model
model = AutoModelForCausalLM.from_pretrained("path_to_your_model")
tokenizer = AutoTokenizer.from_pretrained("path_to_your_model")

@app.route('/')
def home():
    return "Welcome to the Chatbot API!"

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    if user_input:
        inputs = tokenizer(user_input, return_tensors="pt")
        response = model.generate(**inputs)
        bot_response = tokenizer.decode(response[0], skip_special_tokens=True)
        return jsonify({"response": bot_response})
    return jsonify({"error": "No message provided"}), 400

if __name__ == "__main__":
    app.run(debug=True)


ImportError: cannot import name 'nn' from partially initialized module 'torch' (most likely due to a circular import) (/usr/local/lib/python3.11/dist-packages/torch/__init__.py)

In [ ]:
!pip install flask-ngrok



In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return "Ngrok running!"

if __name__ == '__main__':
    app.run(5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [ ]:
!pip install flask-ngrok

from flask import Flask, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok tunnel automatically

@app.route("/")
def home():
    return "Hello from Flask via Ngrok on Colab!"

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [ ]:
# 1. Install and configure ngrok (Run only once in Colab)
!pip install flask-ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# 2. Set your ngrok auth token (Get it from https://dashboard.ngrok.com)
!./ngrok authtoken ngrok config add-authtoken 2wBQXMH0E4x26fRhCcBZELh9mW8_7gHNVAQPMtdkFHN5RmiRT


--2025-04-24 16:39:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 35.71.179.82, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  17.5MB/s    in 0.8s    

2025-04-24 16:39:09 (17.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
NAME:
   authtoken - save authtoken to configuration file

USAGE:
   ngrok authtoken [command options] [arguments...]

DESCRIPTION:
   The authtoken command modifies your configuration file to include
   the specified authtoken. By default, this configuration file is located
   at $HOME/.ngrok2/ngrok.yml

   The ngrok.com service requires that y

In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import numpy as np

app = Flask(__name__)

# Model ko load karen
model = load_model('/content/drive/MyDrive/gpt2_cybercrime_model')

@app.route('/predict', methods=['POST'])
def predict():
    # Input data ko request se lein
    data = request.get_json()
    input_data = np.array(data['input'])  # Yeh aapke input ke format pe depend karega

    # Model ka prediction
    prediction = model.predict(input_data)

    # Output ko return karein
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(debug=True)



UnboundLocalError: cannot access local variable 'weights_store' where it is not associated with a value